<a href="https://colab.research.google.com/github/un21988/CE888-7-project-2/blob/main/Emotion_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.1MB/s 
     |████████████████████████████████| 901kB 17.6MB/s 
     |████████████████████████████████| 3.3MB 22.0MB/s 


In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [5]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[0] for row in csvreader if len(row) > 0]
labels

['0', '1', '2', '3']

In [6]:
import pandas as pd
import urllib.request 
import requests

test_text_url = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/test_text.txt"
val_text_url= "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/emotion/val_text.txt"
test_lable_url= "https://raw.githubusercontent.com/un21988/CE888-7-project-2/main/Data-set/emotion/test_labels.txt"



r = requests.get(test_text_url, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r1 = requests.get(val_text_url, allow_redirects=True)
open('val_text.txt', 'wb').write(r1.content)


r2 = requests.get(test_lable_url, allow_redirects=True)
open('test_lable.txt', 'wb').write(r2.content)






2842

In [7]:
!nvidia-smi

Thu Apr 22 13:13:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:

stream=open("test_text.txt")
tweets=stream.readlines()
stream.close()
print(tweets[1])

@user Interesting choice of words... Are you confirming that governments fund #terrorism? Bit of an open door, but still... 



In [47]:
stream1=open("test_lable.txt")
test_lable=stream1.readlines()
stream1.close()
#print(test_lable)
test_data=[]
print(type(test_lable[1]))
for i in range(0,len(test_lable)):
  temp_data=test_lable[i]
  test_data.append(int(temp_data))
print(test_data)
print(type(test_data[1]))


<class 'str'>
[3, 0, 3, 1, 1, 0, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 1, 0, 0, 0, 0, 3, 3, 2, 2, 1, 1, 0, 3, 1, 3, 1, 0, 0, 0, 0, 2, 2, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 3, 3, 2, 3, 1, 0, 3, 3, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 2, 2, 1, 0, 0, 1, 3, 0, 0, 0, 1, 1, 3, 3, 1, 1, 0, 1, 1, 1, 3, 0, 0, 0, 1, 3, 1, 0, 1, 0, 2, 1, 0, 3, 0, 0, 0, 3, 3, 3, 1, 0, 2, 1, 0, 1, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 1, 1, 1, 2, 3, 3, 0, 3, 3, 0, 0, 3, 2, 0, 0, 0, 1, 2, 0, 0, 3, 0, 1, 3, 1, 3, 0, 3, 0, 2, 1, 1, 1, 0, 3, 0, 1, 0, 0, 2, 1, 2, 0, 0, 0, 1, 1, 1, 0, 0, 3, 3, 0, 3, 3, 3, 3, 0, 0, 2, 3, 2, 1, 1, 0, 0, 3, 1, 0, 0, 0, 1, 1, 0, 0, 3, 3, 1, 3, 2, 1, 1, 3, 0, 0, 3, 0, 2, 3, 3, 3, 3, 0, 2, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 1, 1, 0, 3, 0, 1, 3, 3, 3, 3, 1, 1, 3, 0, 0, 3, 0, 1, 1, 3, 0, 3, 1, 0, 2, 0, 2, 0, 3, 1, 0, 3, 0, 0, 3, 3, 1, 0, 3, 3, 3, 3, 3, 0, 2, 3, 0, 0, 0, 1, 0, 0, 0, 3, 3, 1, 3, 2, 1, 3, 1, 0, 1, 0, 0, 3, 1, 3, 0, 1, 3, 3, 0, 3, 0, 1, 0, 1, 3, 3, 3, 0, 3, 3, 0, 0, 1, 0, 0, 0

In [10]:
stream2=open("val_text.txt")
val_text=stream2.readlines()
stream2.close()
print(val_text[1])

if not then #teamchristine bc all tana has done is provoke her by tweeting shady shit and trying to be a hard bitch begging for a fight 



In [11]:
tweet_preprocess=[]

#print(len(tweets))
for i in range(0,len(tweets)):
  tweet_preprocess.append(preprocess(tweets[i]))
  

print(tweet_preprocess[0])


#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse 



In [12]:
result= []
for i in  range(0,len(tweet_preprocess)):
  result.append(model(tweet_preprocess[i]))
  
 
#print(result)

Streaming output truncated to the last 5000 lines.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequen

In [20]:
print(result)

[['1) 3 0.966', '2) 0 0.0184', '3) 2 0.011', '4) 1 0.0046'], ['1) 2 0.4538', '2) 0 0.2607', '3) 3 0.2034', '4) 1 0.0822'], ['1) 3 0.984', '2) 0 0.007', '3) 1 0.005', '4) 2 0.004'], ['1) 1 0.9216', '2) 2 0.0455', '3) 0 0.0179', '4) 3 0.0151'], ['1) 1 0.5961', '2) 2 0.3472', '3) 3 0.0411', '4) 0 0.0156'], ['1) 0 0.9687', '2) 2 0.0126', '3) 3 0.011', '4) 1 0.0077'], ['1) 3 0.9746', '2) 1 0.0146', '3) 2 0.0057', '4) 0 0.0051'], ['1) 3 0.9759', '2) 0 0.0117', '3) 2 0.0064', '4) 1 0.006'], ['1) 1 0.7872', '2) 2 0.0941', '3) 3 0.0778', '4) 0 0.0409'], ['1) 0 0.9809', '2) 3 0.0087', '3) 1 0.0054', '4) 2 0.005'], ['1) 0 0.9804', '2) 3 0.0101', '3) 2 0.0052', '4) 1 0.0043'], ['1) 3 0.9377', '2) 0 0.0356', '3) 2 0.0176', '4) 1 0.0091'], ['1) 3 0.9643', '2) 2 0.0195', '3) 0 0.011', '4) 1 0.0053'], ['1) 0 0.9763', '2) 2 0.0106', '3) 3 0.0081', '4) 1 0.005'], ['1) 0 0.9121', '2) 2 0.0535', '3) 3 0.0265', '4) 1 0.0079'], ['1) 3 0.9743', '2) 2 0.0102', '3) 0 0.0079', '4) 1 0.0076'], ['1) 1 0.9237', '2

In [21]:

#-------------------Saving the results of the model in CSV file to save time ------------
with open("emotion_model_result.csv", 'w') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerows(result)      

  

In [22]:

final_result =[]
predicted_data=[] # The result we are getting by running the model
for i in  range(0,len(result)):
  final_result.append(min(result[i]))

for i in range(0,len(final_result)):
  extract_data=int(final_result[i][3])
  predicted_data.append(extract_data)
  extract_data= 0
  print("Predicted value: " , predicted_data[i])
  print(type(predicted_data[i]))
  print ("Test value: " , test_data[i])





Streaming output truncated to the last 5000 lines.
Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  1
<class 'int'>
Test value:  2

Predicted value:  1
<class 'int'>
Test value:  1

Predicted value:  2
<class 'int'>
Test value:  2

Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  1
<class 'int'>
Test value:  1

Predicted value:  1
<class 'int'>
Test value:  1

Predicted value:  1
<class 'int'>
Test value:  1

Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  3
<class 'int'>
Test value:  3

Predicted value:  3
<class 'int'>
Test value:  3

Predicted value:  0
<class 'int'>
Test value:  0

Predicted value:  0
<class 'int'>
Test value:  3

Predicted value:  0
<class 'int'>
Test value:  3

Predicted value:  3
<class 'int'>
Test value:  3

Predicted value:  1
<class 'int'>
Test value:  3


In [32]:
print(test_data)

['3\n', '0\n', '3\n', '1\n', '1\n', '0\n', '3\n', '3\n', '3\n', '0\n', '0\n', '3\n', '3\n', '0\n', '0\n', '3\n', '1\n', '0\n', '0\n', '0\n', '0\n', '3\n', '3\n', '2\n', '2\n', '1\n', '1\n', '0\n', '3\n', '1\n', '3\n', '1\n', '0\n', '0\n', '0\n', '0\n', '2\n', '2\n', '0\n', '3\n', '2\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '3\n', '3\n', '2\n', '3\n', '1\n', '0\n', '3\n', '3\n', '2\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '0\n', '3\n', '0\n', '0\n', '0\n', '0\n', '3\n', '2\n', '2\n', '1\n', '0\n', '0\n', '1\n', '3\n', '0\n', '0\n', '0\n', '1\n', '1\n', '3\n', '3\n', '1\n', '1\n', '0\n', '1\n', '1\n', '1\n', '3\n', '0\n', '0\n', '0\n', '1\n', '3\n', '1\n', '0\n', '1\n', '0\n', '2\n', '1\n', '0\n', '3\n', '0\n', '0\n', '0\n', '3\n', '3\n', '3\n', '1\n', '0\n', '2\n', '1\n', '0\n', '1\n', '0\n', '0\n', '1\n', '0\n', '0\n', '0\n', '3\n', '1\n', '0\n', '0\n', '1\n', '1\n', '1\n', '2\n', '3\n', '3\n', '0\n', '3\n', '3\n', '0\n'

In [24]:
print(predicted_data)

[3, 2, 3, 1, 1, 0, 3, 3, 1, 0, 0, 3, 3, 0, 0, 3, 1, 3, 0, 0, 0, 3, 3, 0, 2, 1, 3, 0, 3, 1, 3, 1, 0, 0, 0, 0, 2, 3, 0, 3, 0, 0, 3, 1, 3, 0, 0, 1, 1, 1, 3, 3, 1, 3, 2, 0, 3, 3, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3, 2, 2, 1, 0, 0, 1, 3, 0, 0, 0, 1, 1, 3, 3, 1, 3, 0, 0, 1, 1, 3, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0, 3, 1, 3, 3, 0, 2, 3, 0, 3, 0, 0, 3, 0, 0, 0, 3, 1, 0, 0, 1, 1, 1, 0, 3, 3, 3, 3, 0, 0, 0, 3, 1, 0, 0, 3, 3, 2, 0, 0, 3, 2, 0, 3, 1, 3, 0, 2, 0, 2, 1, 1, 1, 0, 3, 0, 1, 0, 0, 1, 1, 2, 0, 0, 0, 1, 1, 1, 0, 0, 3, 3, 0, 0, 0, 3, 1, 0, 0, 2, 3, 0, 1, 1, 0, 0, 3, 2, 1, 0, 0, 1, 1, 0, 0, 1, 3, 1, 3, 2, 3, 1, 3, 0, 0, 3, 0, 2, 3, 3, 3, 3, 0, 2, 3, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 1, 1, 0, 3, 2, 1, 3, 3, 3, 3, 1, 1, 3, 0, 0, 3, 0, 1, 1, 3, 0, 0, 1, 0, 1, 0, 2, 0, 3, 1, 0, 3, 0, 0, 3, 3, 1, 0, 1, 3, 3, 3, 0, 0, 2, 1, 0, 0, 3, 1, 0, 0, 0, 3, 3, 1, 3, 2, 1, 3, 3, 0, 1, 0, 0, 3, 2, 3, 0, 1, 3, 3, 0, 3, 0, 3, 0, 1, 3, 0, 3, 0, 3, 3, 0, 0, 1, 0, 0, 0, 3, 2, 3, 1, 

In [48]:
def getAccuracy(test_data,predicted_data):
   n = len(test_data) 
   correct = 0
   for x in range(n):
       if test_data[x] == predicted_data[x]:
           correct += 1
   return (correct/n) * 100.0




accuracy = getAccuracy(test_data,predicted_data)
print("we got the accuracy by ", accuracy ,"%")
  

we got the accuracy by  83.46235045742435 %


In [19]:

def model(tweets):

 model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

 text = tweets
 encoded_input = tokenizer(text, return_tensors='tf')
 output = model(encoded_input)
 scores = output[0][0].numpy()
 scores = softmax(scores)

 temp_list=[]
 ranking = np.argsort(scores)
 ranking = ranking[::-1]
 for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    temp_list.append(f"{i+1}) {l} {np.round(float(s), 4)}")
 return temp_list